In [5]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd

## input variable

In [48]:
View_Column_name = "PrevContext_NextContext_PrevTarget_NextTarget_ShapeTarget"
DATE = input()
DATE

'2021-01-14/19-53-56/projector/in_test/09'

## load data

In [49]:
# load csv
df = pd.read_csv("../outputs/{}/tensor.csv".format(DATE), header=None, delimiter="\t")
df_label = pd.read_csv("../outputs/{}/metadata.tsv".format(DATE), delimiter="\t", index_col=0)

In [50]:
data = pd.concat([df_label, df], axis=1)
del df, df_label

## def loss

In [51]:
import scipy.spatial.distance
def cossim(x, y):
    return scipy.spatial.distance.cosine(x, y)

def mean_cossim(X=context_x, Y=target_x):
    tmp = 0.
    for i in range(context_x.shape[0]):
        x = X[i, :]
        y = Y[i, :]
        tmp += cossim(x, y)
    return 1-tmp/context_x.shape[0]

## select vector

In [52]:
vector_dict = {
    "prev_context": None,
    "next_context": None,
    "prev_target": None,
    "next_target": None,
    "now_target": None,
}
for i, name in enumerate(vector_dict.keys()):
     vector_dict[name] = np.array(data.loc[data[View_Column_name]==i].iloc[:, 2:])

## Calculate

In [53]:
tmp = []
for i, (k, v) in enumerate(vector_dict.items()):
    for j, (k_, v_) in enumerate(vector_dict.items()):
        if i<j and k != k_:
            cossim_value = mean_cossim(v, v_)
            print("{} VS {} \t\t {:.5f}".format(k, k_, cossim_value))
            tmp.append([k, k_, cossim_value])

prev_context VS next_context 		 0.99852
prev_context VS prev_target 		 0.99846
prev_context VS next_target 		 0.99510
prev_context VS now_target 		 0.99931
next_context VS prev_target 		 0.99663
next_context VS next_target 		 0.99639
next_context VS now_target 		 0.99952
prev_target VS next_target 		 0.99327
prev_target VS now_target 		 0.99749
next_target VS now_target 		 0.99616


In [54]:
# 20k samples
df = pd.DataFrame(tmp, columns=["vector A", "vector B", "cossim"])
df.sort_values("cossim", ascending=False)

,vector A,vector B,cossim
6,next_context,now_target,0.999515
3,prev_context,now_target,0.999314
0,prev_context,next_context,0.998523
1,prev_context,prev_target,0.998462
8,prev_target,now_target,0.997485
4,next_context,prev_target,0.996626
5,next_context,next_target,0.996385
9,next_target,now_target,0.996163
2,prev_context,next_target,0.995101
7,prev_target,next_target,0.993274


In [47]:
# 2000 samples
df = pd.DataFrame(tmp, columns=["vector A", "vector B", "cossim"])
df.sort_values("cossim", ascending=False)

,vector A,vector B,cossim
3,prev_context,now_target,0.998032
6,next_context,now_target,0.998022
1,prev_context,prev_target,0.997837
5,next_context,next_target,0.997659
9,next_target,now_target,0.997655
2,prev_context,next_target,0.997125
8,prev_target,now_target,0.997001
7,prev_target,next_target,0.996105
0,prev_context,next_context,0.996035
4,next_context,prev_target,0.995904
